<a href="https://colab.research.google.com/github/freoshrew/UHI/blob/main/UHI_GEE_v0_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install geemap and import libraries
geemap is a Python package for interactive mapping with Google Earth Engine. The complete documentation can be found at https://geemap.org/.

ee is the Python API package for Google Earth Engine. It must be imported and initialized for each new Python session and script.


In [ ]:
# Install geemap package
!pip install -U geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 7.6 MB/s 
     |████████████████████████████████| 198 kB 62.2 MB/s 
     |████████████████████████████████| 130 kB 46.9 MB/s 
     |████████████████████████████████| 3.0 MB 45.5 MB/s 
     |████████████████████████████████| 8.8 MB 38.2 MB/s 
     |████████████████████████████████| 1.3 MB 55.3 MB/s 
     |████████████████████████████████| 99 kB 10.0 MB/s 
     |████████████████████████████████| 98 kB 5.5 MB/s 
     |████████████████████████████████| 46 kB 3.4 MB/s 
     |████████████████████████████████| 1.2 MB 41.0 MB/s 
     |████████████████████████████████| 3.4 MB 41.1 MB/s 
     |████████████████████████████████| 128 kB 47.2 MB/s 
     |████████████████████████████████| 1.6 MB 48.2 MB/s 
     |████████████████████████████████| 9.7 MB 44.9 MB/s 
     |████████████████████████████████| 344 kB 59.4 MB/s 
     |███████████████████████████████

In [ ]:
# Import libraries
from IPython.display import Javascript
import geemap.geemap as geemap
import ee

In [ ]:
# Authenticate to the Earth Engine servers
ee.Authenticate()

# Initialize the API
#ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=3iaBcRkNJZofFPAnah1VuOOq78rMLYDXn1LUH8dmFu8&tc=xbNJ4PLskXjrfuGpplG2rmDtfwlRoCHcXKboQEzEeMY&cc=KU6S3xgSkwFxkua02bsIzui3Vb1FAUEZFqk-txln-3I

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AdQt8qhLJsOz-fMSypgajbraerV4IcNz2vegFi3xmUgW-uaAgkyZ5iDYRl8

Successfully saved authorization token.


# Create an interactive map

In [ ]:
Map = geemap.Map()
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))
Map

AttributeError: ignored

# Load Earth Engine datasets

In [ ]:
# Add Sentinel-2 image collection from Earth Engine data catalogue
ic = ee.ImageCollection('COPERNICUS/S2_SR')

# Check how many images are in the image collection
print('Size:', ic.size().getInfo()) # Too many elements - need to filter!

# Filtering

In [ ]:
# Center the map on Perth and create a rectangle geometry
Map = geemap.Map(center=[-32, 116], zoom=12)
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))
Map

In [ ]:
# First create a rectangle geometry around Perth
# perth = ee.Geometry.Rectangle([[115.71981587648668, -31.91568688885592],
#                               [115.96906819582262, -32.10898710342787]])
perth = Map.user_roi

# Filter the image collection using the Perth geometry
filteredPerth = ic.filterBounds(perth)

# Then we'll check the size again
print('Size when filtered by bounds:', filteredPerth.size().getInfo())

In [ ]:
# Now let's also filter by date and properties and chain them together
filtered = (ic.filterBounds(perth)
            # Filter for images between these two dates
            .filterDate('2019-01-01','2020-12-31')
            # Filter for images with less than 30% cloud cover
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 0.3)))

print('Size with multiple filters:', filtered.size().getInfo())

In [ ]:
# Finally we'll look at a red, green, blue mosaic (bands 4, 3 and 2 respectively)
image = filtered.select('B4','B3','B2').mosaic()

# Add the image to the map using an arbitrary stretch between 0 and 2500 
# (does not change the data itself it is just for visualisation purposes)
Map = geemap.Map(center=[-32, 116], zoom=12)
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))
Map.addLayer(image, {'min':0,'max':2500}, 'RGB')
Map

# Reducing

In [ ]:
# Find the min and max values within our three bands ('B4', 'B3' and 'B2) within our geometry
thisReducer = ee.Reducer.minMax()
stats = image.reduceRegion(thisReducer, perth, 1000).getInfo()
print('Stats:', stats)

In [ ]:
# Create an image representing the mean value of each pixel through the time series
meanReducer = ee.Reducer.mean()
imageMean = filtered.reduce(meanReducer).select(['B4_mean', 'B3_mean', 'B2_mean'])

Map = geemap.Map(center=[-32, 116], zoom=12)
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))
Map.addLayer(imageMean, {'min':0, 'max':2500}, 'RGB (Mean)')
Map

# Raster Algebra (Band Math)

In [ ]:
# Calculate Normalised Difference Vegetation Index (NDVI)
imageMean = filtered.reduce(meanReducer)
imageRed = imageMean.select('B4_mean')
imageNIR = imageMean.select('B8_mean')
imageNDVI = (imageNIR.subtract(imageRed)).divide(imageNIR.add(imageRed))

Map = geemap.Map(center=[-32, 116], zoom=12)
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))
Map.addLayer(imageNDVI, {min:0.0, max:0.8}, 'NDVI')
Map

# Data Visualisation

In [ ]:
# Visualise a single band
Map = geemap.Map(center=[-32, 116], zoom=12)
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))
Map.addLayer(imageRed, {'min':0, 'max':3000}, 'Red Band')
Map

In [ ]:
# Visualise true and false colour imagery
image = (filtered.reduce(meanReducer)
      # Rename bands
      .select(['B2_mean','B3_mean','B4_mean','B8_mean'], ['B2','B3','B4','B8']))

Map = geemap.Map(center=[-32, 116], zoom=12)
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))
Map.addLayer(image, {'min':0, 'max':3000, 'bands':['B4','B3','B2']}, 'True Colour')
Map.addLayer(image, {'min':0, 'max':3000, 'bands':['B8','B4','B3']}, 'False Colour')
Map

In [ ]:
# Create a thematic map of NDVI categories
# No Veg (0): NDVI 0.0 - 0.2
# Low Veg (1): NDVI 0.2 - 0.4
# Medium Veg (2): NDVI 0.4 - 0.6
# High Veg (3): NDVI 0.6 +

classes = ee.Image(0)
lowVeg = imageNDVI.gt(0.2) # returns a binary image where (0 <= 0.2, 1 > 0.2)
medVeg = imageNDVI.gt(0.4) # returns a binary image where (0 <= 0.4, 1 > 0.4)
highVeg = imageNDVI.gt(0.6) # returns a binary image where (0 <= 0.6, 1 > 0.6)

classes = classes.add(lowVeg).add(medVeg).add(highVeg)

Map = geemap.Map(center=[-32, 116], zoom=12)
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))
Map.addLayer(classes, {'palette':['999900','004400','009900','00ff00']}, 'Veg Classes')
Map

In [ ]:
# Visualise a feature collection
fc = ee.FeatureCollection("FAO/GAUL/2015/level0")

Map = geemap.Map()
display(Javascript('''
  google.colab.widgets.installCustomManager('https://ssl.gstatic.com/colaboratory-static/widgets/colab-cdn-widget-manager/6a14374f468a145a/manager.min.js');
'''))
Map.addLayer(fc, {}, 'Country Boundaries')
Map